# import libraries

In [1]:
import os
import glob
import pandas as pd

# import and explore data

**import data**

In [4]:
raw_data_path = "/home/phillip/Dokumente/repositories/ds1_weather_data_ger/data/raw_data/raw_data_stations/stations_basedata.txt"

df_data_raw = pd.read_fwf(raw_data_path, encoding='ISO-8859-1')
print(df_data_raw)

      Stations_id von_datum bis_datum  \
0     ----------- --------- ---------   
1             00001 19370101 19860630   
2             00003 18910101 20110331   
3             00011 19800901 20200930   
4             00044 19690101 20230613   
...                               ...   
1346          19607 19720501 20010430   
1347          19617 19410101 20061231   
1348          19631 19060101 19691231   
1349          19647 19510101 20051031   
1350          19774 19710819 19940228   

     Stationshoehe geoBreite geoLaenge Stationsname Bundesland  
0     ------------- --------- --------- ------------...         
1     478     47.8413    8.8493 Aach                ...         
2     202     50.7827    6.0941 Aachen              ...         
3     680     47.9736    8.5205 Donaueschingen (Land...         
4     44     52.9336    8.2370 Großenkneten         ...         
...                                                 ...         
1346  615     50.6952   13.4972 Cämmerswalde-Rausche

**explore data**

In [6]:
#show number of columns and rows
df_data_raw .shape

# show column names and data types
df_data_raw.info() 

df_data_raw.head() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1351 entries, 0 to 1350
Data columns (total 2 columns):
 #   Column                                                     Non-Null Count  Dtype 
---  ------                                                     --------------  ----- 
 0   Stations_id von_datum bis_datum                            1351 non-null   object
 1   Stationshoehe geoBreite geoLaenge Stationsname Bundesland  1351 non-null   object
dtypes: object(2)
memory usage: 21.2+ KB


,Stations_id von_datum bis_datum,Stationshoehe geoBreite geoLaenge Stationsname Bundesland
0,----------- --------- ---------,------------- --------- --------- ------------...
1,00001 19370101 19860630,478 47.8413 8.8493 Aach ...
2,00003 18910101 20110331,202 50.7827 6.0941 Aachen ...
3,00011 19800901 20200930,680 47.9736 8.5205 Donaueschingen (Land...
4,00044 19690101 20230613,44 52.9336 8.2370 Großenkneten ...
